In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data_pca.csv")
df.head()

,Customer_ID,Z1,Z2,Z3,Z4,Z5,Z6,Z7,Z8,Z9,...,Z19,Z20,Z21,Z22,Z23,Z24,Z25,Z26,Z27,Credit_Score
0,3392,-1.432269,0.093852,-2.278188,-1.407816,-0.135649,-1.996803,0.275828,-0.648048,1.313756,...,0.667774,0.094029,-0.093014,-1.100186,-1.494958,1.076635,-1.688978,0.175915,-0.819142,2
1,3392,-1.698989,0.418354,-1.785871,-1.361343,-0.131089,-1.898032,0.406715,-0.559605,1.350804,...,0.595757,-0.238172,0.142565,-1.179211,-1.534768,0.976853,-1.743069,0.064623,-0.666969,2
2,3392,-1.501422,0.479586,-1.432461,-1.360819,-0.066890,-1.854806,0.282160,-0.416728,1.355014,...,1.041039,0.708216,-0.509633,-1.109486,-1.522738,1.003116,-1.690114,0.156681,-0.533545,2
3,3392,-1.682258,0.101027,-2.322815,-1.393282,-0.149784,-1.995335,0.318837,-0.708591,1.318216,...,0.514263,-0.515451,0.413239,-1.106747,-1.530995,0.928847,-1.573458,0.124982,-0.749807,2
4,3392,-1.582761,0.624981,-1.505005,-1.382160,-0.070191,-1.849546,0.218287,-0.380822,1.363299,...,1.189984,1.238533,-0.868051,-1.067568,-1.493789,1.074852,-1.702390,0.253632,-0.557135,2


In [3]:
X = df.drop(["Customer_ID", "Credit_Score"], axis=1)
y = df.Credit_Score
index = df.Customer_ID

In [4]:
X.head()

,Z1,Z2,Z3,Z4,Z5,Z6,Z7,Z8,Z9,Z10,...,Z18,Z19,Z20,Z21,Z22,Z23,Z24,Z25,Z26,Z27
0,-1.432269,0.093852,-2.278188,-1.407816,-0.135649,-1.996803,0.275828,-0.648048,1.313756,-0.713133,...,0.575968,0.667774,0.094029,-0.093014,-1.100186,-1.494958,1.076635,-1.688978,0.175915,-0.819142
1,-1.698989,0.418354,-1.785871,-1.361343,-0.131089,-1.898032,0.406715,-0.559605,1.350804,-0.727694,...,0.592985,0.595757,-0.238172,0.142565,-1.179211,-1.534768,0.976853,-1.743069,0.064623,-0.666969
2,-1.501422,0.479586,-1.432461,-1.360819,-0.066890,-1.854806,0.282160,-0.416728,1.355014,-0.817413,...,0.454585,1.041039,0.708216,-0.509633,-1.109486,-1.522738,1.003116,-1.690114,0.156681,-0.533545
3,-1.682258,0.101027,-2.322815,-1.393282,-0.149784,-1.995335,0.318837,-0.708591,1.318216,-0.665006,...,0.666473,0.514263,-0.515451,0.413239,-1.106747,-1.530995,0.928847,-1.573458,0.124982,-0.749807
4,-1.582761,0.624981,-1.505005,-1.382160,-0.070191,-1.849546,0.218287,-0.380822,1.363299,-0.871936,...,0.384913,1.189984,1.238533,-0.868051,-1.067568,-1.493789,1.074852,-1.702390,0.253632,-0.557135


In [5]:
from sklearn.model_selection import GroupShuffleSplit


gss = GroupShuffleSplit(n_splits=2, test_size=0.3, random_state=42)
for train_index, test_index in gss.split(X, groups=index):
    print("Train:", train_index, "Test:", test_index)
    X_train = X.iloc[train_index, :]
    y_train = y[train_index]
    X_test = X.iloc[test_index, :]
    y_test = y[test_index]

Train: [   16    17    18 ... 99965 99966 99967] Test: [    0     1     2 ... 99997 99998 99999]
Train: [    0     1     2 ... 99989 99990 99991] Test: [    8     9    10 ... 99997 99998 99999]


In [12]:
param_grid = {"C": [0.1, 0.25, 0.5, 0.75, 1], "l1_ratio": [0.1, 0.25, 0.5, 0.75, 1]}

In [13]:
from sklearn.linear_model import LogisticRegression

logit = LogisticRegression(solver="saga", max_iter=1000, penalty="elasticnet", class_weight="balanced")

In [14]:
from sklearn.model_selection import GridSearchCV
CV_logit = GridSearchCV(estimator=logit, param_grid=param_grid, cv= 5, scoring='f1_macro', n_jobs=8)
CV_logit.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=LogisticRegression(class_weight='balanced',
                                          max_iter=1000, penalty='elasticnet',
                                          solver='saga'),
             n_jobs=8,
             param_grid={'C': [0.1, 0.25, 0.5, 0.75, 1],
                         'l1_ratio': [0.1, 0.25, 0.5, 0.75, 1]},
             scoring='f1_macro')

In [15]:
CV_logit.best_params_

{'C': 0.1, 'l1_ratio': 1}

In [16]:
y_train_pred = CV_logit.predict(X_train)
y_test_pred = CV_logit.predict(X_test)

In [17]:
from sklearn.metrics import accuracy_score

print(f"Train Accuracy: {accuracy_score(y_train, y_train_pred)}")
print(f"Test Accuracy: {accuracy_score(y_test, y_test_pred)}")

Train Accuracy: 0.5938142857142857
Test Accuracy: 0.5879333333333333


In [ ]:
from sklearn.model_selection import GridSearchCV
CV_logit = GridSearchCV(estimator=logit, param_grid=param_grid, cv= 5, scoring='f1_macro')
CV_logit.fit(X_train, y_train)